Following this guide for understanding and building: https://www.youtube.com/watch?v=TCH_1BHY58I

Doing mostly by memory of process: 

In [2]:
import torch 
import torch.nn.functional as F
import matplotlib.pyplot

In [4]:
words = open('names.txt').read().splitlines()

In [12]:
# Build stoi and i to itos
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [49]:
input_length = 3

def create_data(data):
    x = []
    y = []

    for word in words:
        context = [0] * input_length
        for char in word + '.': 
            indx = stoi[char]
            x.append(context)
            y.append(indx)
            context = context[1:] + [indx]

    x = torch.tensor(x)
    y = torch.tensor(y)
    return x, y

import random 
random.shuffle(words)

length = len(words)

xtr,ytr = create_data(words[:int((length*0.8))])
xval,yval = create_data(words[int((length*0.8)):int((length*0.9))])
xtest,ytest = create_data(words[int(length*0.9):])

In [57]:
total_chars = len(stoi)
num_embeddings = 10
middle_nodes = 200

C =  torch.randn(total_chars, num_embeddings)
W1 = torch.randn(num_embeddings * input_length, middle_nodes)
b1 = torch.randn(middle_nodes)
W2 = torch.randn(middle_nodes, total_chars)
b2 = torch.randn(total_chars)

parameters = [C,W1,b1,W2,b2]

for p in parameters:
    p.requires_grad = True

In [62]:
# Batch Forward pass
num_trainings = 20000
batch_size = 32
for i in range(num_trainings):
    mini = torch.randint(0,xtr.shape[0], (batch_size,))
    emb = C[xtr[mini]].view(mini.shape[0], -1)
    h = torch.tanh(emb @ W1 + b1)
    logits = h @ W2 + b2
    counts = logits.exp()
    prob = counts/counts.sum(dim=1,keepdims=True)
    loss = -prob[torch.arange(emb.shape[0]), ytr[mini]].log().mean()
    # Backprop
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data += -0.001 * p.grad
print(loss.item())

1.8861414194107056


In [63]:
emb = C[xval].view(xval.shape[0], -1)
h = torch.tanh(emb @ W1 + b1)
logits = h @ W2 + b2
counts = logits.exp()
prob = counts/counts.sum(dim=1,keepdims=True)
loss = -prob[torch.arange(xval.shape[0]), yval].log().mean()
loss.item()

2.124830722808838